In [28]:
from super_gradients.training import models
from super_gradients.common.object_names import Models

yolo_nas_pose = models.get("yolo_nas_pose_l", pretrained_weights="coco_pose", checkpoint_path=".\\Models\\yolo_nas_pose_l_coco_pose.pth").cuda()

[2024-04-01 23:37:53] WARNING - checkpoint_utils.py - :warning: The pre-trained models provided by SuperGradients may have their own licenses or terms and conditions derived from the dataset used for pre-training.
 It is your responsibility to determine whether you have permission to use the models for your use case.
 The model you have requested was pre-trained on the coco_pose dataset, published under the following terms: https://cocodataset.org/#termsofuse
[2024-04-01 23:37:53] INFO - checkpoint_utils.py - License Notification: YOLO-NAS-POSE pre-trained weights are subjected to the specific license terms and conditions detailed in 
https://github.com/Deci-AI/super-gradients/blob/master/LICENSE.YOLONAS-POSE.md
By downloading the pre-trained weight files you agree to comply with these terms.
[2024-04-01 23:37:54] INFO - checkpoint_utils.py - Successfully loaded pretrained weights for architecture yolo_nas_pose_l
[2024-04-01 23:37:54] INFO - checkpoint_utils.py - Successfully loaded mo

In [29]:
import torch
torch.cuda.is_available()

True

In [30]:
import torch
import os
import pathlib
import re
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import cv2

In [31]:
from tensorflow.keras.preprocessing.image import img_to_array
class ImageToArrayPreprocessor:
    def __init__(self, dataFormat=None):
        self.dataFormat = dataFormat

    def preprocess(self, image):
        return img_to_array(image, data_format=self.dataFormat)

class SimplePreprocessor:
    def __init__(self, width, height, inter=cv2.INTER_AREA):
        self.width = width
        self.height = height
        self.inter = inter

    def preprocess(self, image):
        return cv2.resize(image, (self.width, self.height), interpolation=self.inter)

In [32]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array
from scipy.stats import mode

class_names = ['barbell biceps curl',
'bench press',
'chest fly machine',
'deadlift',
'decline bench press',
'hammer curl',
'hip thrust',
'incline bench press',
'lat pulldown',
'lateral raise',
'leg extension',
'leg raises',
'normal',
'plank',
'pull Up',
'push-up',
'romanian deadlift',
'russian twist',
'shoulder press',
'squat',
't bar row',
'tricep Pushdown',
'tricep dips']

image_width, image_height = 64, 64
model_path = './/Models//Resnet_skeleton.keras'
sp = SimplePreprocessor(image_width, image_height)
iap = ImageToArrayPreprocessor()
model_predict = load_model(model_path)
def predict_image(frames, no_batch_size=32):
  preprocessedFrames = []
  for frame in frames:
      frame = sp.preprocess(frame)  # Thay đổi kích thước
      frame = iap.preprocess(frame)  # Chuyển đổi sang mảng
      preprocessedFrames.append(frame)

  frames = np.array(preprocessedFrames, dtype="float") / 255.0  # Chuẩn hóa
  predictions = model_predict.predict(frames, batch_size=no_batch_size)  # Dự đoán
  # Lấy chỉ số lớp với xác suất cao nhất cho mỗi mẫu
  class_indices = np.argmax(predictions, axis=1)
  print(class_indices)
  counts = np.bincount(class_indices)
  if np.argmax(counts) == 12:
        counts_temp = counts.copy()
        counts_temp[12] = -1
        most_common_class_idx = np.argmax(counts_temp)
  else:
      most_common_class_idx = np.argmax(counts)

  print(class_names[most_common_class_idx])
  return most_common_class_idx

In [33]:
import matplotlib.pyplot as plt
import numpy as np
import cv2
from super_gradients.training.utils.visualization.detection import draw_bbox
from super_gradients.training.utils.visualization.pose_estimation import PoseVisualization

def process_single_image(image_prediction, largest_bbox_idx):
    image = image_prediction.image
    pose_data = image_prediction.prediction

    # Lấy box của pose có chỉ số largest_bbox_idx
    bbox = pose_data.bboxes_xyxy[largest_bbox_idx]
    x1, y1, x2, y2 = bbox

    # Tính toán tâm và kích thước mới của box
    center_x, center_y = (x1 + x2) / 2, (y1 + y2) / 2
    width, height = x2 - x1, y2 - y1

    # Mở rộng box ra 20%
    new_width, new_height = width * 1.2, height * 1.2
    new_x1, new_y1 = center_x - new_width / 2, center_y - new_height / 2
    new_x2, new_y2 = center_x + new_width / 2, center_y + new_height / 2

    # Điều chỉnh để không vượt qua ranh giới của ảnh
    new_x1, new_y1 = max(new_x1, 0), max(new_y1, 0)
    new_x2, new_y2 = min(new_x2, image.shape[1]), min(new_y2, image.shape[0])

    # Crop ảnh
    cropped_image = image[int(new_y1):int(new_y2), int(new_x1):int(new_x2)]

    # Tạo một bản sao của ảnh để vẽ lên
    blank_image = np.zeros_like(cropped_image)

    # Lấy chỉ một pose tương ứng với hộp giới hạn lớn nhất
    single_pose = np.expand_dims(pose_data.poses[largest_bbox_idx], axis=0)

    # Cập nhật tọa độ của pose sau khi crop
    single_pose[:, :, 0] -= new_x1
    single_pose[:, :, 1] -= new_y1

    # Vẽ khung xương cho pose đó
    skeleton_image = PoseVisualization.draw_poses(
        image=blank_image,
        poses=single_pose,
        boxes=None,  # Không hiển thị hộp giới hạn
        scores=None,  # Bỏ qua điểm số cho ví dụ này
        is_crowd=None,
        edge_links=pose_data.edge_links,
        edge_colors=pose_data.edge_colors,
        keypoint_colors=pose_data.keypoint_colors,
        joint_thickness=5,
        keypoint_radius=20
    )
    return skeleton_image

In [34]:
def process_video(video_path, frame_interval=200):
  cap = cv2.VideoCapture(video_path)

  if not cap.isOpened():
      print("Error: Could not open video.")
      exit()
  frames = []
  frame_count = 0
  current_time = 0
  while True:
      ret, frame = cap.read()
      if not ret:
          break
      current_time = cap.get(cv2.CAP_PROP_POS_MSEC)

      if current_time >= frame_count * frame_interval:
        results = yolo_nas_pose.to('cuda').predict(frame, conf=.25, fuse_model=False)
        bboxes_xyxy = results.prediction.bboxes_xyxy
        if len(bboxes_xyxy) > 0:
          # Tính diện tích của các hộp giới hạn nếu có
          areas = (bboxes_xyxy[:, 2] - bboxes_xyxy[:, 0]) * (bboxes_xyxy[:, 3] - bboxes_xyxy[:, 1])
          index_of_largest_bbox = np.argmax(areas)
          # Xử lý và hiển thị ảnh với khung xương của hộp giới hạn lớn nhất
          skeleton_image = process_single_image(results, index_of_largest_bbox)
          frames.append(skeleton_image)
  cap.release()
  predicted_class = predict_image(frames)
  return predicted_class

In [35]:
import os
from sklearn.metrics import confusion_matrix, f1_score, classification_report
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

def process_folders(root_dir):
    labels = []
    video_names = []
    for folder in os.listdir(root_dir):
        video_dir = os.path.join(root_dir, folder)
        if os.path.isdir(video_dir):
            for video_file in os.listdir(video_dir):
                video_names.append(video_file)
                video_path = os.path.join(video_dir, video_file)
                print(video_path)
                if os.path.isfile(video_path):
                    predicted_label = process_video(video_path)
                    predicted_label = class_names[predicted_label]
                    labels.append(predicted_label)
    return labels, video_names

# Example usage
root = '.\\Dataset'
labels, video_names = process_folders(root)

.\Data1\bench press\bench press_1.mp4
11/11 [==============================] - 4s 287ms/step
[18  1 10 10 10 10 14 22 10 14 14 14 10 18 18  1  6  6 14 14 14 14 14 14
 14 14 18 14 12 14 14 14 10 12  1  1 12 10 10  8 14  8 10 10 10 10 10 10
 10 10 10 10 18 18 18 18 18 18 18 18 18 18 18 18 18 18 18 10 10 10 14 10
 14 12 17 10 10 12 12 14 18  1  8 14 10 10  1 10 10 10 10 10 18 18 18 18
 18 18 18 18 18 18 10 10 10 15 10 10 10 10 10 14 12 10 10 18 18  8 14 14
  8 14 14 14 14 14 14 14 14  8 14 18 14 18 18 18 18 10 12 10 10 10 10 10
 10 10 10 10 10 10 18 10 10 18 18 18 18 18 18 18 18 18 18 18 18 18 18 18
 18 12 18 18 18 12 12 18 18 18 18 10 18 12 14 14 14 14 14 14 14 14 12  1
 12 14 14 14 14 14 10 10 10 10 10 10 10 10 18 18 18 18  8 12 18 18 12 10
 12 18 18 18 18 18 18 18 18 18 18 18 18 10  1  1 12 10 18 18 18 10 12 10
 10 12 14 12 14 12 14 12 12 14 14 14 14 14 14 14 14 12 12 12 10 18 18 18
 18 18 12 18 18 12 12 12 18 18 18 18 18 18 18 18 18 18 18 18 18 18 12  1
 12 12 18 18 10 12 12 12 10 12 

8/8 [==============================] - 2s 276ms/step
[ 3 20 12  3 10 10  3  6  6  6 10 10 10 10  6  6  6  6  4  6  6  6 10  6
  6  6  6  6  6  6  6  6  6 10  6 10 10 10 10 10  3 10 10 10 10 16  3  1
 10 10 12 12 12 12 10 10 10  6  6 10  6  1  1  6  6  4  4  6  6  4  6  6
  6  6  6  6  6  6  6  6  6  6  6  6  6  4  6  6  6  6  6  6  6  6 10 10
 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10  6  6  6  6  4
  6  6  6  6  6  6  6  6  6 10 10 10 10 10 10 13 10 13 10 10 10 10 10 10
 10 10 10 10 10 10 10  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6 10
 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10  6 10
  6 10  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6 10 10 10 10 10 10 10
 10 10 10 12 10 12 10 10 10 10 10 10 13 10 10 10  6  6  6  3 12 12]
leg extension


In [36]:
import csv

def write_arrays_to_csv(file_path, array1, array2):
    # Open the CSV file for writing
    with open(file_path, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        
        # Write the header row
        writer.writerow(['video', 'Dự đoán'])
        
        # Write the data from the arrays to the file
        for i in range(max(len(array1), len(array2))):
            value1 = array1[i] if i < len(array1) else ''
            value2 = array2[i] if i < len(array2) else ''
            writer.writerow([value1, value2])
# Call the function to create the CSV file and write the arrays to it
write_arrays_to_csv('data.csv', video_names, labels)